In [21]:
import json

from torch.ao.nn.quantized.functional import threshold

# open and read content job_skills
with open("job_skill.json", "r", encoding='utf-8') as f:
    job_skills = json.load(f)

job_skills = [job for job in job_skills.values()]
import pandas as pd
import json

# open and read content job_skills
with open("job_skill.json", "r", encoding='utf-8') as f:
    job_skills = json.load(f)

all_job_skills = [skill["skill"] for profession in job_skills.values() for skill in profession]
unique_job_skills = list(set(all_job_skills))

print(unique_job_skills)
print(len(all_job_skills), len(unique_job_skills))

['kotlin', 'sap', 'azure functions', 'python', 'github', 'nosql', 'airflow', 'ci cd', 'vba', 'unreal engine', 'livedata', 'animation basics', 'confluence', 'cloud security', 'ssrs', 'navigation component', 'java ee', 'prototyping', '1с документооборот', 'data modeling', 'cassandra', 'sas', 'blueprints', 'user research', 'yii', 'r', 'databricks', 'golang', 'rxswift', 'azure data factory', 'git', 'helpdesk', 'ssis', 'viper', 'kubernetes', 'penetration testing', 'linq', 'ab testing', 'puppet', 'data cleaning', 'elk stack', 'multithreading', 'quantitative research', 'azure blob', 'webpack', 'cybersecurity', 'rxjava', 'tensorflow', 'postman', 'persona creation', 'windows', 'feature engineering', 'graphics programming', 'scrum', 'keras', 'angular', 'presentation', 'test case', '1c предприятие', 'branding basics', 'message queues', 'sql', 'laravel', 'express.js', 'data pipelines', 'network configuration', 'elasticsearch', 'power bi', 'message brokers', 'data visualization', 'etl tools', 'scss

In [22]:
import pandas as pd
import ast

df = pd.read_csv("courses_final.csv")

df['skills'] = df['skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

all_courses_skills = df['skills'].explode()
unique_course_skills = all_courses_skills.dropna().unique().tolist()

print(unique_course_skills)
print(len(all_courses_skills), len(unique_course_skills))


['NumPy', 'Data Manipulation', 'File Management', 'Data Structures', 'Application Programming Interface (API)', 'Data Analysis', 'Programming Principles', 'Web Scraping', 'Restful API', 'Object Oriented Programming (OOP)', 'Computer Programming', 'Jupyter', 'Python Programming', 'Data Import/Export', 'Pandas (Python Package)', 'Development Environment', 'Computational Thinking', 'Integrated Development Environments', 'Debugging', 'Algorithms', 'Problem Management', 'Network Protocols', 'Database Design', 'SQL', 'Data Collection', 'Data Visualization Software', 'Data Processing', 'Data Modeling', 'Test Driven Development (TDD)', 'Software Testing', 'Unit Testing', 'Plotly', 'Docker (Software)', 'Scripting', 'Git (Version Control System)', 'Data Ethics', 'Agile Methodology', 'Flask (Web Framework)', 'Web Development', 'Matplotlib', 'Statistical Modeling', 'Supervised Learning', 'Predictive Modeling', 'Regression Analysis', 'Data Cleansing', 'Data Pipelines', 'Descriptive Statistics', 'Fe

In [23]:
import re

def clean_skill(skill: str) -> str:
    cleaned = re.sub(r'\s*\(.*?\)', '', skill)
    return cleaned.strip().lower()

cleaned_course_skills = list(set([clean_skill(skill) for skill in unique_course_skills]))
print(cleaned_course_skills)

['sales', 'divide and conquer', 'embedded software', 'semantic web', 'data entry', 'language translation', 'nodes', 'user story', 'technical analysis', 'data as a service', 'network topology', 'statistical programming', 'prediction', '3d computer graphics', 'pesticides', 'securities trading', 'data storage', 'prototyping', 'blood pressure', 'linear discriminant analysis', 'full-stack web development', 'data processing', 'programming concepts', 'event-driven programming', 'localization', 'database permissions', 'user research', 'extensible markup language', 'statistical hypothesis testing', 'peer review', 'cron', 'computer-aided design', 'voice assistant technology', 'data analysis', 'language model', 'online service provider', 'usability', 'version control', 'analysis', 'software design', 'tensorflow', 'test automation', 'c', 'ecmascript 2015', 'customer service', 'microsoft 365', 'computer security', 'human readable', 'persuasive communication', 'applicant tracking systems', 'openshif

In [24]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/multilingual-e5-large')
# model = SentenceTransformer('multilingual-e5-large')

In [25]:
course_embeddings = model.encode(cleaned_course_skills, normalize_embeddings=True)
job_embeddings = model.encode(unique_job_skills, normalize_embeddings=True)


In [26]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(job_embeddings, course_embeddings)

In [70]:
import numpy as np

skill_map = {}
threshold = 0.90
for i, prof_skill in enumerate(unique_job_skills):

    top_indices = [
        idx for idx in np.argsort(similarity_matrix[i])[::-1]
            if threshold < similarity_matrix[i][idx] and not np.isclose(similarity_matrix[i][idx], 1.0)
    ][:3]
    if top_indices:
        skill_map[prof_skill] = [
            (cleaned_course_skills[idx], round(float(similarity_matrix[i][idx]), 2))
            for idx in top_indices
        ]

print(skill_map)
for prof_skill, matches in skill_map.items():
    print(f"Skill from job: {prof_skill}")
    for course_skill, score in matches:
        print(f"  --> {course_skill} (accuracy: {score})")
    print()

{'airflow': [('dataflow', 0.9)], 'ci cd': [('ci/cd', 0.94)], 'unreal engine': [('unity engine', 0.91), ('3d engine', 0.9)], 'ab testing': [('a/b testing', 0.92), ('medical testing', 0.91), ('system testing', 0.9)], 'data cleaning': [('data cleansing', 0.96)], 'multithreading': [('multitasking', 0.93)], 'azure blob': [('microsoft azure', 0.91)], 'persona creation': [('persona development', 0.94), ('persona', 0.91)], 'windows': [('microsoft windows', 0.9)], 'graphics programming': [('computer programming', 0.92), ('statistical programming', 0.9), ('python programming', 0.9)], 'presentation': [('presentations', 0.94), ('data presentation', 0.94)], 'express.js': [('react.js', 0.94), ('node.js', 0.91), ('vue.js', 0.91)], 'network configuration': [('network administration', 0.92), ('network routing', 0.91), ('network operating system', 0.91)], 'client management': [('resource management', 0.91), ('problem management', 0.91), ('information management', 0.91)], 'api': [('cloud api', 0.9)], 'us

In [71]:
from collections import defaultdict

reverse_skill_map = defaultdict(list)

for prof_skill, course_matches in skill_map.items():
    for course_skill, score in course_matches:
        reverse_skill_map[course_skill].append((prof_skill, score))

final_map = {}
for course_skill, skills_scores in reverse_skill_map.items():
    skills_scores.sort(key=lambda x: x[1], reverse=True)
    most_relevant_skill = skills_scores[0][0]
    final_map[course_skill] = most_relevant_skill

print(final_map)

{'dataflow': 'mlflow', 'ci/cd': 'ci cd', 'unity engine': 'unreal engine', '3d engine': 'unreal engine', 'a/b testing': 'ui testing', 'medical testing': 'manual testing', 'system testing': 'manual testing', 'data cleansing': 'data cleaning', 'multitasking': 'multithreading', 'microsoft azure': 'azure', 'persona development': 'persona creation', 'persona': 'persona creation', 'microsoft windows': 'windows', 'computer programming': 'graphics programming', 'statistical programming': 'graphics programming', 'python programming': 'graphics programming', 'presentations': 'presentation', 'data presentation': 'presentation', 'react.js': 'express.js', 'node.js': 'express.js', 'vue.js': 'express.js', 'network administration': 'network configuration', 'network routing': 'network configuration', 'network operating system': 'network configuration', 'resource management': 'user management', 'problem management': 'project management', 'information management': 'client management', 'cloud api': 'rest a

In [73]:
# with open("reverse_skill_map.json", 'w', encoding='utf-8') as file:
#     json.dump(final_map, file, ensure_ascii=False, indent=4)

Key как записано названи е у нас в курсах, value -- на что надо заменить для ранжирования

In [35]:
import ast

df = pd.read_csv("courses_final.csv")

def normalize_and_map(skill_list):
    normalized = []
    for skill in skill_list:
        cleaned = re.sub(r'\s*\(.*?\)', '', skill).strip().lower()
        mapped = reverse_skill_map.get(cleaned, cleaned)
        normalized.append(mapped if isinstance(mapped, str) else mapped[0])
    return normalized

df['skills'] = df['skills'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

df['mapped_skills'] = df['skills'].apply(normalize_and_map)

print("Пример замены:")
print(df[['skills', 'mapped_skills']].head(15))

df.to_csv('courses_normalized_skills.csv', index=False, encoding='utf-8-sig')

Пример замены:
                                               skills  \
0   [NumPy, Data Manipulation, File Management, Da...   
1   [Development Environment, Computational Thinki...   
2   [Network Protocols, Database Design, SQL, Appl...   
3   [Debugging, Test Driven Development (TDD), Int...   
4   [Plotly, Debugging, Test Driven Development (T...   
5   [Scripting, Data Analysis, Object Oriented Pro...   
6   [Data Import/Export, Matplotlib, Pandas (Pytho...   
7   [Development Environment, Programming Principl...   
8   [Unit Testing, Debugging, Image Analysis, Pyth...   
9   [Python Programming, Computer Programming, Dat...   
10  [Unit Testing, Debugging, Infrastructure as Co...   
11  [Scripting, Scripting Languages, Programming P...   
12  [Unsupervised Learning, Applied Machine Learni...   
13  [SQL, Query Languages, Jupyter, Databases, Dat...   
14  [Program Development, Application Development,...   

                                        mapped_skills  
0   [numpy, data

### Normalization
1. to lower case
2. delete any info in brackets as (API)
3. delete any -

In [10]:
import re
def clean_skill(skill: str) -> str:
    skill = skill.lower()
    skill = re.sub(r'#+', '', skill)
    skill = skill.replace("-", " ")
    skill = re.sub(r'\s+', ' ', skill)
    skill = re.sub(r'[^\w\s]', '', skill)
    return skill.strip()

def clean_skills(cell: str) -> str:
    skills = [clean_skill(s) for s in cell.split(",")]
    return ", ".join(skills)

In [15]:
import json

mapping = []
with open("reverse_skill_map.json", "r", encoding="utf-8") as f:
    mapping = json.load(f)

def map_skills_list(cell: str) -> str:
    skills = [clean_skill(s) for s in cell.split(",")]
    mapped = [mapping.get(s, s) for s in skills]
    return ", ".join(mapped)

In [16]:
import pandas as pd

def process_skills_list(skills: list[str], mapping: dict) -> tuple[list[str], list[str]]:
    cleaned = [clean_skill(s) for s in skills]
    mapped = [mapping.get(s, s) for s in cleaned]
    return mapped

In [20]:
import ast

df = pd.read_csv("courses_final.csv")

df["skills"] = df["skills"].apply(ast.literal_eval)
df["skills"] = df["skills"].apply(lambda x: process_skills_list(x, mapping))

df.to_csv("courses_final_mapped.csv", index=False)

In [27]:
import requests, time

error_link = []

for course_url in df.url:
    try:
        print(f"Checking: {course_url}")
        response = requests.get(course_url, timeout=10)

        if response.status_code not in [200, 201]:
            print(f"❌ Broken link: {course_url} (status {response.status_code})")
            error_link.append(course_url)
        else:
            print(f"✅ OK: {course_url}")
    except Exception as e:
        print(f"⚠️ Error for {course_url}: {e}")
        error_link.append(course_url)

Checking: https://www.coursera.org/learn/python-for-applied-data-science-ai
⚠️ Error for https://www.coursera.org/learn/python-for-applied-data-science-ai: HTTPSConnectionPool(host='www.coursera.org', port=443): Read timed out. (read timeout=10)
Checking: https://www.coursera.org/learn/python-crash-course
⚠️ Error for https://www.coursera.org/learn/python-crash-course: HTTPSConnectionPool(host='www.coursera.org', port=443): Read timed out. (read timeout=10)
Checking: https://www.coursera.org/specializations/python
✅ OK: https://www.coursera.org/specializations/python
Checking: https://www.coursera.org/learn/programming-in-python
✅ OK: https://www.coursera.org/learn/programming-in-python
Checking: https://www.coursera.org/professional-certificates/microsoft-python-developer
✅ OK: https://www.coursera.org/professional-certificates/microsoft-python-developer
Checking: https://www.coursera.org/learn/get-started-with-python
✅ OK: https://www.coursera.org/learn/get-started-with-python
Checki

In [31]:
df = df[~df['url'].isin(error_link)]

# rewrite courses
df.to_csv('courses.csv', index=False, encoding='utf-8')
